In [1]:
#import modules
import numpy as np
from pyspark.sql.types import *
from pyspark.sql.functions import *

# start
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import StorageLevel

# sc = SparkContext(master="local[4]")
appName = "MapReduce in Spark"
spark = SparkSession.builder.master("local[4]") \
        .appName(appName) \
        .config("spark.driver.memory", "15g") \
        .config('spark.executor.memory','15g') \
        .getOrCreate()
spark

In [2]:
#define our schema
flightSchema = StructType([
  StructField("Time", FloatType()),
  StructField("V1", FloatType()),
  StructField("V2", FloatType()),
  StructField("V3", FloatType()),
  StructField("V4", FloatType()),
  StructField("V5", FloatType()),
  StructField("V6", FloatType()),
  StructField("V7", FloatType()),
  StructField("V8", FloatType()),
  StructField("V9", FloatType()),
  StructField("V10", FloatType()),
  StructField("V11", FloatType()),
  StructField("V12", FloatType()),
  StructField("V13", FloatType()),
  StructField("V14", FloatType()),
  StructField("V15", FloatType()),
  StructField("V16", FloatType()),
  StructField("V17", FloatType()),
  StructField("V18", FloatType()),
  StructField("V19", FloatType()),
  StructField("V20", FloatType()),
  StructField("V21", FloatType()),
  StructField("V22", FloatType()),
  StructField("V23", FloatType()),
  StructField("V24", FloatType()),
  StructField("V25", FloatType()),
  StructField("V26", FloatType()),
  StructField("V27", FloatType()),
  StructField("V28", FloatType()),
  StructField("Amount", FloatType()),
  StructField("Class", FloatType()),
])

csv = spark.read.csv('D:/mrjob/creditcard.csv',schema=flightSchema, header=True)

In [3]:
#divide data, xx % for training, xx% for testing
dividedData = csv.randomSplit([0.8, 0.2])
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 227764 ; Testing data rows: 57043


In [4]:
trainlist = []
for i in trainingData.collect():
    trainlist.append(tuple(i))

In [ ]:
# trainlist[442]

In [5]:
testlist = []
for i in testingData.collect():
    testlist.append(tuple(i))

In [ ]:
# testlist[100]

In [6]:
import math
import time
import numpy as np
import pandas as pd

start_time = time.time()
k = 3
distance = 0
distancelist = []
resultlist = []
train_class_list = []
order_list = []

for m in range(0,500): #each test row === len(testlist) ===
    for j in range(0,len(trainlist)): #each train row ==== len(trainlist) ====
        distance = 0
        for i in range(28): #each train feature 
            distance += math.pow((trainlist[j][i] - testlist[m][i]),2)
        distance = math.pow(distance,0.5)
        distancelist.append(distance)
        train_class = trainlist[j][30]
        train_class_list.append(train_class)
    rdd1 = spark.sparkContext.parallelize(distancelist)
    rdd2 = spark.sparkContext.parallelize(train_class_list)
    rdd1.persist()
    rdd2.persist()
    rdd3 = rdd1.zip(rdd2).map(lambda x : (x[0],x[1])).takeOrdered(k, key=lambda x:x[0])
    rdd4 = spark.sparkContext.parallelize(rdd3).values().sum()
    result = 0
    if rdd4 <= k/2:
        result = 0.0
    else:
        result = 1.0
    resultlist.append(result)
    rdd1.unpersist()
    rdd2.unpersist()
print("finish in ",time.time() - start_time)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 124.0 failed 1 times, most recent failure: Lost task 0.0 in stage 124.0 (TID 482) (192.168.182.104 executor driver): java.net.SocketException: Software caused connection abort: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:111)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:155)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:126)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:295)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.s

In [8]:
resultlist

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]